# Analizando el modelo Sugarscape.
**A continuación se presenta un análisis de los datos arrojados por algunos experimentos realizados con el modelo de Sugarscape. Veremos la relación de la variable de dotación inicial (maximum-sugar-endowment y minimum-sugar-endowment) y las métricas del índice GINI, la cantidad de decesos por hambre (starvation), la riqueza per cápita y la cantidad promedio de cambios de azúcar tras aplicar las políticas de recaudación y redistribución. Los mapas de calor de cada experimentación pueden ser consultados a detalle en el otro notebook adjuntado.**
## Importando bibliotecas

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re
from pathlib import Path
sns.set()

# Métodos auxiliares

In [2]:
def limpiar_dataframe(df):
    limpiado = df.drop(columns='visualization')
    limpiado = limpiado.drop(limpiado[limpiado['[step]'] == 0].index) # aquellas runs que no se iniciaron
    limpiado = limpiado.drop(columns='[run number]')
    limpiado = limpiado.drop(columns='initial-population') #poblacion fija
    limpiado = limpiado.drop(columns='[step]') # en este punto, todas las runs son validas y llegaron a la it final
    try:
        limpiado = limpiado.drop(columns='taxation')
        limpiado = limpiado.drop(columns='redistribution')
    except:
        return limpiado
        
    return limpiado
    
def mapa_calor(df,medida,titulo,ruta):
    datos_aplanados = df.pivot(index="minimum-sugar-endowment", columns="maximum-sugar-endowment", values=medida)
    plt.figure(figsize=(12, 8)) 
    sns.heatmap(datos_aplanados, cmap='viridis',annot=False)
    plt.text(0, -0.5, titulo, fontsize=12, color='black', ha='left')
    plt.savefig(ruta,dpi=300, bbox_inches='tight')
    plt.show()
    

# Variables utiles

In [3]:
# Las diferentes implementaciones de politicas seran medidas segun su reduccion del
# indice gini, reduccion de decesos por hambruna, riqueza per capita mas alta y
# diferencias de riquezas menos marcadas
tipos_analisis = {'Indice Gini':'avg-gini',
                  'Riqueza per cápita':'avg-productivity',
                  'Diferencia de riquezas entre un tick y otro':'avg-diff',
                  'Decesos por hambruna' : 'starvation'
                 }
# Diccionario con listas de dataframes pivotados para su analisis
datos_analisis = {'Indice Gini': [],
                  'Riqueza per cápita': [],
                  'Diferencia de riquezas entre un tick y otro':[],
                  'Decesos por hambruna' : []
                 }
# Una lista donde se almacenaran los nombres de cada politica implementada
nombres_politicas = []

# Mapas de calor sobre el índice GINI, decesos por hambruna y riqueza per cápita

In [4]:
# Observando los resultados del modelo base sin politicas
datos_base = pd.read_csv('datasets/simple.csv')
datos_base = limpiar_dataframe(datos_base)
#for llave in tipos_analisis.keys():
#    mapa_calor(datos_base, tipos_analisis[llave], llave, 'figuras/'+llave)



## Cargando los datos arrojados por la implementación de diferentes politicas de recaudación redistribución
Los archivos siguen el patron tipo_recaudación-tipo_redistribución.csv

In [5]:

# patron util para recuperar el tipo de politica de recaudacion y redistribucion
carpeta_datasets = Path('datasets/')
patron = r"(\w+)-(\w+)\.csv$"

for archivo in carpeta_datasets.glob("*.csv"):
    # dataset correspondiende a la implementacion de una politica de recaudacion combinada con una de redistribucion
    df = pd.read_csv(archivo)
    df = limpiar_dataframe(df)
    match = re.match(patron, archivo.name)
    print('cargando: ', archivo.name)
    if match:
        # nombre de la politica de recaudacion y redistribucion
        recaudacion = match.group(1)
        redist = match.group(2)
        nombre = recaudacion +' '+ redist
        nombres_politicas.append(nombre)
        # dataframes pivotados para cada analisis(gini, riqueza, decesos, diferencias) de cada experimento
        for llave in tipos_analisis.keys():
            medida = tipos_analisis[llave]
            datos_aplanados = df.pivot(index="minimum-sugar-endowment", columns="maximum-sugar-endowment", values=medida)
            datos_analisis[llave].append(datos_aplanados)
            


cargando:  lineal-dirigida.csv
cargando:  dinamica-uniforme.csv
cargando:  lineal-dinamica.csv
cargando:  uniforme-lineal.csv
cargando:  lineal-uniforme.csv
cargando:  dinamica-dinamica.csv
cargando:  lineal-lineal.csv
cargando:  simple.csv
cargando:  dinamica-lineal.csv
cargando:  uniforme-uniforme.csv
cargando:  uniforme-dinamica.csv


# Analizando los resultados
Ahora que ya tenemos los datos con las métricas arrojados por cada uno de los experimentos con las políticas, analicemos los mejores resultados 

In [9]:
def menos_es_mejor(iniciales: pd.DataFrame, pivotados: list) -> (pd.DataFrame,pd.DataFrame):
    """
    Metodo auxiliar que compara celda por celda los valores
    de cada dataframe en la lista de pivotados y mantiene
    el valor mas bajo. 
    iniciales : dataframe con resultados por defecto sin politicas
    pivotados : lista de dataframes con datos pivotados
    returns: una tupla con el dataframe de resultados y de etiquetas
    respectivamente. Notese que cada etiqueta corresponde a un indice
    de la lista nombres_politicas
    """
    base = iniciales.copy()
    etiquetas = iniciales.copy()
    etiquetas[~ etiquetas.isnull()] = -1
    for idx in base.index:
        for col  in base.columns:
            if base.loc[idx,col] == base.loc[idx,col]: # evita valores NaN
                for i in range(len(pivotados)):
                    mejor_valor = base.loc[idx,col]
                    politica_actual = pivotados[i]
                    valor_propuesto = politica_actual.loc[idx,col]
                    if valor_propuesto < mejor_valor:
                        base.loc[idx,col] = valor_propuesto
                        etiquetas.loc[idx,col] = i
    return (base,etiquetas)
    
def mas_es_mejor(iniciales: pd.DataFrame, pivotados: list) -> (pd.DataFrame,pd.DataFrame):
    """
    Metodo auxiliar que compara celda por celda los valores
    de cada dataframe en la lista de pivotados y mantiene
    el valor mas alto. 
    iniciales : dataframe con resultados por defecto sin politicas
    pivotados : lista de dataframes con datos pivotados
    returns: una tupla con el dataframe de resultados y de etiquetas
    respectivamente. Notese que cada etiqueta corresponde a un indice
    de la lista nombres_politicas
    """
    base = iniciales.copy()
    etiquetas = iniciales.copy()
    etiquetas[~ etiquetas.isnull()] = -1
    for idx in base.index:
        for col  in base.columns:
            if base.loc[idx,col] == base.loc[idx,col]: # evita valores NaN
                for i in range(len(pivotados)):
                    mejor_valor = base.loc[idx,col]
                    politica_actual = pivotados[i]
                    valor_propuesto = politica_actual.loc[idx,col]
                    if valor_propuesto > mejor_valor:
                        base.loc[idx,col] = valor_propuesto
                        etiquetas.loc[idx,col] = i
    return (base,etiquetas)

def muestra_resultados(etiquetas:pd.DataFrame):
    """
        Metodo que imprime las mejores politicas
        segun el rango de azucar inicial
        etiquetas: dataframe con las etiquetas
        de las mejores politicas segun el analisis
    """
    for idx in etiquetas.index:
        for col in etiquetas.columns:
            if pd.notna(etiquetas.loc[idx,col]):
                indice = int(etiquetas.loc[idx,col])
                politica = nombres_politicas[indice]
                print(f"La mejor politica con una dotacion minima de {idx} y una maxima de {col} es {politica}")

## Mejor politica para reducir el indice Gini segun el rango de riqueza inicial

In [10]:
# Los valores base del indice gini del modelo sin politicas
base_gini = datos_base.pivot(index="minimum-sugar-endowment", columns="maximum-sugar-endowment", values='avg-gini')
pivotados_gini = datos_analisis['Indice Gini']
resultados, etiquetas = menos_es_mejor(iniciales=base_gini, pivotados=pivotados_gini)
muestra_resultados(etiquetas)

La mejor politica con una dotacion minima de 0 y una maxima de 10 es lineal uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 20 es lineal uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 30 es lineal uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 40 es lineal uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 50 es lineal uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 60 es lineal uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 70 es lineal uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 80 es lineal uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 90 es lineal uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 100 es lineal uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 110 es lineal uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 

## Mejor politica para reducir los decesos por hambruna

In [11]:
# Los valores base de la cantidad de decesos por hambruna del modelo sin politicas
base_starv = datos_base.pivot(index="minimum-sugar-endowment", columns="maximum-sugar-endowment", values='starvation')
pivotados_starv= datos_analisis['Decesos por hambruna']
resultados, etiquetas = menos_es_mejor(iniciales=base_starv, pivotados=pivotados_starv)
muestra_resultados(etiquetas)

La mejor politica con una dotacion minima de 0 y una maxima de 10 es dinamica uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 20 es lineal uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 30 es dinamica uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 40 es dinamica uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 50 es dinamica uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 60 es dinamica uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 70 es lineal uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 80 es uniforme uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 90 es uniforme uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 100 es lineal uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 110 es uniforme uniforme
La mejor politica con una dotacion minima de 0 

## Mejor politica para maximizar la riqueza per cápita

In [13]:

base_riqueza = datos_base.pivot(index="minimum-sugar-endowment", columns="maximum-sugar-endowment", values='avg-productivity')
pivotados_riqueza = datos_analisis['Riqueza per cápita']
resultados, etiquetas = mas_es_mejor(iniciales=base_riqueza, pivotados=pivotados_riqueza)
muestra_resultados(etiquetas)

La mejor politica con una dotacion minima de 0 y una maxima de 10 es uniforme dinamica
La mejor politica con una dotacion minima de 0 y una maxima de 20 es uniforme dinamica
La mejor politica con una dotacion minima de 0 y una maxima de 30 es lineal lineal
La mejor politica con una dotacion minima de 0 y una maxima de 40 es dinamica uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 50 es uniforme lineal
La mejor politica con una dotacion minima de 0 y una maxima de 60 es uniforme lineal
La mejor politica con una dotacion minima de 0 y una maxima de 70 es dinamica uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 80 es lineal uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 90 es dinamica uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 100 es lineal uniforme
La mejor politica con una dotacion minima de 0 y una maxima de 110 es lineal lineal
La mejor politica con una dotacion minima de 0 y una maxi